In [107]:
import pandas as pd
import numpy as np 

In [140]:
## load dataset
baskets = []
with open("../data/ItemBasketDataset/transactions.dat", encoding='utf-8', errors='replace') as f:
    for line in f.readlines():
        basket = line.split(" ")
        clean_basket = []
        for i, itemStr in enumerate(basket): 
            if (i != len(basket)-1):
                clean_basket.append(int(itemStr))
        baskets.append(set(clean_basket))
        
baskets = baskets[0:10000]

In [141]:
len(baskets)

10000

In [142]:
class Apriori: 
    def __init__(this, baskets, N):
        this.baskets = baskets
        this.C = [set()] * N ## Initialize with empty set at 0 because we will not use C0. 
        this.L = [set()] * N ## Initialize with empty set at 0 because we will not use L0. 
        this.S = {}
        this.N = N
        
        
    def computeSupport(this, k): 
        """ walks through the buckets and counts each candidate occurence, saves to F[k] """
        count = {}
        for candidateRepr in this.C[k]:
            candidate = eval(candidateRepr)
            for basket in this.baskets: 
                if (candidate.issubset(basket)): ## If candidate occurs in basket
                    if (candidateRepr not in count): 
                        count[candidateRepr] = 1
                    count[candidateRepr] += i
        for candidateRepr in this.C[k]:
            support = count[candidateRepr]/len(this.baskets) ## Support is expressed as the fraction of transaction in which the candidate occurs
            this.S[candidateRepr] = support
                
    def createCandidates(this, k): 
        """ creates candidates C[k] """
        
        return set()
                
    def filterCandidates(this, k, s): 
        """ filters out the candidates which do not have support in the original baskets """
        frequent = set()
        for candidateRepr in this.C[k]: 
            if this.S[candidateRepr] > s:
                frequent.add(candidateRepr)
        
        return frequent
    
        

## Pipeline
- Compute item support
- Let candidates C1 be all items
- Construct L1 by filtering out candidates with low support
- for k --> N:
    - Construct new candidates Ck based on Ck-1 and C1
    - Construct Lk by filtering out candidates with low support 

In [143]:
## Hyperparams
s = 0.01 # Support threshold
N = 10 # the largest sought itemset



In [144]:
A = Apriori(baskets, N)
k = 1
## Let candidates C1 be the set of all items
for basket in baskets: 
    for item in basket: 
        A.C[1].add(repr(set([item]))) #Every candidate is considered a set

# Compute support of all candidates in C[1]
A.computeSupport(k = k)

## Construct L1 by filtering out candidates with low support
A.L[1] = A.filterCandidates(k = k, s = s)

In [145]:
print("Found {} singleton candidates, out of which {} had support > {}".format(len(A.C[1]), len(A.L[1]), s))

Found 866 singleton candidates, out of which 813 had support > 0.01


In [ ]:
k += 1 ## Increase k to 2

while k <= N
    # Construct new candidates Ck based on Ck-1 and C1
    A.C[k] = # 
    # Construct Lk by filtering out candidates with low support
    A.L[k] = A.filterCandidates(k = 1, s = s)
    
    k += 1

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10])

False

In [70]:
repr(set([1, 2, 3, 7]))

'{1, 2, 3, 7}'

In [72]:
repr(set([1, 2, 7, 3, 3]))

'{1, 2, 3, 7}'